# Crawl, load, and split D&C podcasts from the Church of Jesus Christ of Latter-day Saints

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

In [ ]:
from datetime import datetime
import json
import os
from typing import Iterator
from urllib.parse import urljoin, urlparse

from bs4 import BeautifulSoup, Tag
from tqdm import tqdm

from models.crawl_utils import get_page, save_page
from models.load_dc_podcasts import load_dc_podcasts
from models.load_utils import Loader, load_docs_from_jsonl, save_docs_to_jsonl
from models.split_model import MarkdownSyntacticEmbeddingSplitter

## Crawl

In [ ]:
# config
host = 'https://doctrineandcovenantscentral.org/church-history-matters-podcast/'
source = 'dc_podcasts'  # use this name in the directories
crawl_dir = f'../data/raw/{source}'  # base_dir is now crawl_dir
bs_parser = 'html.parser'
delay_seconds = 5

if not os.path.exists(crawl_dir):
    os.makedirs(crawl_dir)
    

In [ ]:
def extract_series_hrefs(host, html):
    hrefs = []

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Find all div tags with the class 'views-field-title'
    link_tags = soup.find_all('div', {'data-column-clickable': True})

    # Extract href attribute from each anchor tag and append to the list
    for link in link_tags:
        href = link.get('data-column-clickable')
        if href:
            if href.startswith(host):
                path_components = urlparse(href).path.split('/')
                # print(path_components)
                if len(path_components) == 3:
                    if not href in hrefs:
                        hrefs.append(href)

    return hrefs

In [ ]:
def extract_links(host, html):
    hrefs = []

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Find all div tags with the class 'views-field-title'
    link_tags = soup.find_all('div', {'data-column-clickable': True})

    # Extract href attribute from each anchor tag and append to the list
    for link in link_tags:
        href = link.get('data-column-clickable')
        if href:
            if href.startswith(host):
                hrefs.append(href)
                # path_components = urlparse(href).path.split('/')
                # print(path_components)
                # if len(path_components) == 4:
                #    if not href in hrefs:
                #        hrefs.append(href)

    return hrefs

In [ ]:
def get_path(url):
    path_components = urlparse(url).path.split('/')
    # print(path_components)
    return os.path.join(crawl_dir, f"{path_components[-1]}.json")

In [ ]:
status_code, html = get_page(host, delay_seconds)
if status_code != 200:
    print("Error!", status_code)
series_hrefs = extract_series_hrefs(host, html)
print(series_hrefs[:5])
len(series_hrefs)

In [ ]:
dc_podcasts_pages = []

for series_link in series_hrefs:
    status_code, html = get_page(series_link, delay_seconds)
    if status_code != 200:
        print("Error!", status_code)
    episodes_hrefs = extract_links(host, html)
    print(series_link, len(episodes_hrefs), episodes_hrefs[:3])
    if len(episodes_hrefs) != 0:
        dc_podcasts_pages.extend(episodes_hrefs)
    
print(dc_podcasts_pages[:5])
len(dc_podcasts_pages)

In [ ]:
for url in dc_podcasts_pages:
    if url.endswith('/'):
        url = url[:-1]
    path_file =  get_path(url)
    print(path_file)
    if os.path.exists(path_file):
        continue
    status_code, html = get_page(url, delay_seconds)
    if status_code != 200:
        print("Error!", status_code , url)
        continue
    
    save_page(path_file,url,html)
    
print("End")

## Load

In [ ]:
# config
# input_dir is now crawl_dir, and output_dir is now load_dir, and output_filename is now load_filename
load_dir = f'../data/load/{source}/'
today = datetime.today().strftime('%Y-%m-%d')
load_filename = os.path.join(load_dir, f"{today}.jsonl")

if not os.path.exists(load_dir):
    os.makedirs(load_dir)

In [ ]:
loader = Loader(load_dc_podcasts, crawl_dir)
docs = loader.load(verbose=True)
len(docs)

In [ ]:
print("metadata: ", docs[0].metadata)
print()
print("content: ", docs[0].page_content)

In [ ]:
save_docs_to_jsonl(docs, load_filename)

## Split

In [ ]:
# configure
# input_path is now load_filename, output_dir is now split_dir, and output filename is now split_filename
split_dir = f'../data/split/{source}/'
today = datetime.today().strftime('%Y-%m-%d')
# output filename is now split_filename
split_filename = os.path.join(split_dir, f"{today}.jsonl")

if not os.path.exists(split_dir):
    os.makedirs(split_dir)

In [ ]:
docs = load_docs_from_jsonl(load_filename)
len(docs)

In [ ]:
text_splitter = MarkdownSyntacticEmbeddingSplitter()

In [ ]:
splits = text_splitter.split_documents(docs, verbose=True)
len(splits)

In [ ]:
for ix, split in enumerate(splits[:10]):
    print(ix, split.metadata)
    print(split.page_content)
    print("\n!!! SPLIT !!!\n")

In [ ]:
save_docs_to_jsonl(splits, split_filename)